In [1]:
import cProfile
from abc import ABC, abstractmethod

In [2]:
my_arr = [4, 5, 6, [4, 7], 7, 4, [2, 8], 7, [4, 7, [3, 8, 5]], 9, 7, 8,
          [4, 9, [3, 9, [4, 8, [3, 6, 7, 3], 3], 5], 4], 9]

In [3]:
class AbstractFlatten(ABC):
    """
    Abstract class for sorting
    """
    @classmethod
    @abstractmethod
    def flatten(arr: list):
        """Method to unpack/flatten the nested array"""
        pass

    @classmethod
    @abstractmethod
    def run(n: int = 50, print_output: bool = True):
        """CHECK THIS!! !!!!!! !!!!!! !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"""
        pass

    @classmethod
    def run_benchmark(cls, n: int = 50):
        for _ in range(n):
            cls.run(print_output=False)

In [4]:
class NonRecursiveFlatten(AbstractFlatten):
    @classmethod
    def flatten(cls, arr=my_arr):
        nested = True
        while nested:
            new = []
            nested = False
            for i in arr:
                if isinstance(i, list):
                    new.extend(i)
                    nested = True
                else:
                    new.append(i)
            arr = new
        return arr

    @classmethod
    def run(cls, n: int = 50, print_output: bool = True):
        for i in range(n):
            global my_arr
            my_arr.append([1, [1, 1, [486, 453, 342, [3653, 3543, 76757, [2, 5, 3, 4]]]]])
            flattened_array = NonRecursiveFlatten.flatten()
            if i > 20 and print_output:
                print(f'{i} - {flattened_array}')
        my_arr = [4, 5, 6, [4, 7], 7, 4, [2, 8], 7, [4, 7, [3, 8, 5]], 9, 7, 8,
                  [4, 9, [3, 9, [4, 8, [3, 6, 7, 3], 3], 5], 4], 9]

In [5]:
NonRecursiveFlatten.run()

21 - [4, 5, 6, 4, 7, 7, 4, 2, 8, 7, 4, 7, 3, 8, 5, 9, 7, 8, 4, 9, 3, 9, 4, 8, 3, 6, 7, 3, 3, 5, 4, 9, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453,

In [6]:
class RecursiveFlatten(AbstractFlatten):
    @classmethod
    def flatten(cls, arr=my_arr):
        new_list = []
        for i in arr:
            if isinstance(i, list):
                new_list.extend(RecursiveFlatten.flatten(i))
            else:
                new_list.append(i)
        return new_list

    @classmethod
    def run(cls, n: int = 50, print_output: bool = True):
        for i in range(n):
            global my_arr
            my_arr.append([1, [1, 1, [486, 453, 342, [3653, 3543, 76757, [2, 5, 3, 4]]]]])
            flattened_array = RecursiveFlatten.flatten()
            if i > 20 and print_output:
                print(f'{i} - {flattened_array}')
        my_arr = [4, 5, 6, [4, 7], 7, 4, [2, 8], 7, [4, 7, [3, 8, 5]], 9, 7, 8,
                  [4, 9, [3, 9, [4, 8, [3, 6, 7, 3], 3], 5], 4], 9]

In [7]:
RecursiveFlatten.run()

21 - [4, 5, 6, 4, 7, 7, 4, 2, 8, 7, 4, 7, 3, 8, 5, 9, 7, 8, 4, 9, 3, 9, 4, 8, 3, 6, 7, 3, 3, 5, 4, 9, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453, 342, 3653, 3543, 76757, 2, 5, 3, 4, 1, 1, 1, 486, 453,

In [8]:
%timeit NonRecursiveFlatten.run(print_output=False)

7.85 ms ± 83.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit RecursiveFlatten.run(print_output=False)

5.64 ms ± 37 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
cProfile.run('NonRecursiveFlatten.run_benchmark()', sort='ncalls')

         10025054 function calls in 1.910 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  5010000    0.388    0.000    0.388    0.000 {built-in method builtins.isinstance}
  4367500    0.316    0.000    0.316    0.000 {method 'append' of 'list' objects}
   645000    0.072    0.000    0.072    0.000 {method 'extend' of 'list' objects}
     2500    1.130    0.000    1.906    0.001 767210615.py:2(flatten)
       50    0.004    0.000    1.910    0.038 767210615.py:17(run)
        1    0.000    0.000    1.910    1.910 254704589.py:19(run_benchmark)
        1    0.000    0.000    1.910    1.910 <string>:1(<module>)
        1    0.000    0.000    1.910    1.910 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [11]:
cProfile.run('RecursiveFlatten.run_benchmark()', sort='ncalls')

         5350054 function calls (4705054 primitive calls) in 1.209 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2350000    0.181    0.000    0.181    0.000 {built-in method builtins.isinstance}
  1707500    0.132    0.000    0.132    0.000 {method 'append' of 'list' objects}
647500/2500    0.818    0.000    1.205    0.000 445115463.py:2(flatten)
   645000    0.075    0.000    0.075    0.000 {method 'extend' of 'list' objects}
       50    0.004    0.000    1.209    0.024 445115463.py:12(run)
        1    0.000    0.000    1.209    1.209 254704589.py:19(run_benchmark)
        1    0.000    0.000    1.209    1.209 <string>:1(<module>)
        1    0.000    0.000    1.209    1.209 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [12]:
%load_ext line_profiler

`%lprun -f RecursiveFlatten.run RecursiveFlatten.run(print_output=False)`

вызывает ошибку:

`Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.`

#### Здесь без lprun, который у меня не работает, тяжело определить, скорее всего рекурсивная функция работает быстрее как раз из-за многократного рекурсивного вызова, плюс в ней .append меньше раз вызывается, .isinstance тоже меньше раз вызывается. В целом, думаю, чем больше количество итераций с добавлением массива с глубиной 5 вложений, как я добавляю в методе run(), тем больше лишних проходов нерекурсивной функции по этим вышеуказанным методам, что будет все сильнее замедлять ее по отношению к рекурсивной

In [13]:
%load_ext memory_profiler

In [14]:
%%writefile run_nonrecursive_flatten.py

from memory_profiler import profile


def nonrecursive_flatten(arr):
    nested = True
    while nested:
        new = []
        nested = False
        for i in arr:
            if isinstance(i, list):
                new.extend(i)
                nested = True
            else:
                new.append(i)
        arr = new
    return arr


@profile
def run_nonrecursive_flatten():
    array = []
    for i in range(1000):
        array.append([1, [1, 1, [486, 453, 342, [3653, 3543, 76757, [2, 5, 3, 4]]]]])
        result = nonrecursive_flatten(array)
    return result


if __name__ == '__main__':
    run_nonrecursive_flatten()

Writing run_nonrecursive_flatten.py


In [15]:
!python run_nonrecursive_flatten.py

Filename: C:\Users\Мария\PycharmProjects\aleksandr_smolin_python_hw\hw3_profiling\run_nonrecursive_flatten.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    20     41.6 MiB     41.6 MiB           1   @profile
    21                                         def run_nonrecursive_flatten():
    22     41.6 MiB      0.0 MiB           1       array = []
    23     43.5 MiB      0.1 MiB        1001       for i in range(1000):
    24     43.5 MiB      0.3 MiB        1000           array.append([1, [1, 1, [486, 453, 342, [3653, 3543, 76757, [2, 5, 3, 4]]]]])
    25     43.5 MiB      1.6 MiB        1000           result = nonrecursive_flatten(array)
    26     43.5 MiB      0.0 MiB           1       return result




In [16]:
%%writefile run_recursive_flatten.py

from memory_profiler import profile


def recursive_flatten(arr):
    new_list = []
    for i in arr:
        if isinstance(i, list):
            new_list.extend(recursive_flatten(i))
        else:
            new_list.append(i)
    return new_list


@profile
def run_recursive_flatten():
    array = []
    for i in range(1000):
        array.append([1, [1, 1, [486, 453, 342, [3653, 3543, 76757, [2, 5, 3, 4]]]]])
        result = recursive_flatten(array)
    return result


if __name__ == '__main__':
    run_recursive_flatten()

Writing run_recursive_flatten.py


In [17]:
!python run_recursive_flatten.py

Filename: C:\Users\Мария\PycharmProjects\aleksandr_smolin_python_hw\hw3_profiling\run_recursive_flatten.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    15     41.7 MiB     41.7 MiB           1   @profile
    16                                         def run_recursive_flatten():
    17     41.7 MiB      0.0 MiB           1       array = []
    18     42.5 MiB      0.0 MiB        1001       for i in range(1000):
    19     42.5 MiB      0.0 MiB        1000           array.append([1, [1, 1, [486, 453, 342, [3653, 3543, 76757, [2, 5, 3, 4]]]]])
    20     42.5 MiB      0.8 MiB        1000           result = recursive_flatten(array)
    21     42.5 MiB      0.0 MiB           1       return result




#### Непримечательное увеличение памяти в большом диапазоне при добавлении каждый раз по вложенному списку с глубиной 5 уровней вложения, в рекурсивной функции оно меньше